In [1]:
%load_ext autoreload
%autoreload 2
import re
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
from transformers import AdamW
from transformers import AutoModel, BertTokenizerFast
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import sys
sys.path.append('../')
from process_data import *
from models.transformer_cnn import *
from training import train
from evaluation import evaluate
import matplotlib.pyplot as plt

In [2]:
df_zoo_train, df_zoo_test = load_data("../data")
df_zoo_train.head()

,Record_id,Attribute_name,y_act,total_vals,num_nans,%_nans,num_of_dist_val,%_dist_val,mean,std_dev,...,mean_stopword_total,stdev_stopword_total,mean_char_count,stdev_char_count,mean_whitespace_count,stdev_whitespace_count,mean_delim_count,stdev_delim_count,is_list,is_long_sentence
0,33,Area,categorical,21477,0,0.0,174,0.810169,0.000000,0.000000,...,0.2,0.4,10.0,4.816638,0.4,0.8,0.4,0.8,False,False
1,33,Area Code,categorical,21477,0,0.0,174,0.810169,125.449411,72.866452,...,0.0,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,False,False
2,33,Element,categorical,21477,0,0.0,2,0.009312,0.000000,0.000000,...,0.0,0.0,4.0,0.000000,0.0,0.0,0.0,0.0,False,False
3,33,Element Code,categorical,21477,0,0.0,2,0.009312,5211.687154,146.816661,...,0.0,0.0,4.0,0.000000,0.0,0.0,0.0,0.0,False,False
4,33,Item,categorical,21477,0,0.0,115,0.535457,0.000000,0.000000,...,0.8,0.4,19.6,2.244994,2.0,0.0,2.0,0.0,False,False


In [3]:
train_data = preprocess(df_zoo_train, " [SEP] ")
test_data = preprocess(df_zoo_test, " [SEP] ")
train_data.head()

,text,features,label
0,Area [SEP] Afghanistan [SEP] Albania [SEP] Alg...,"[1.058161729698188, -0.49520412160069144, 0.0,...",1.0
1,Area Code [SEP] 2 [SEP] 3 [SEP] 4 [SEP] 7 [SEP] 8,"[1.058161729698188, -0.49520412160069144, 0.0,...",1.0
2,Element [SEP] Food [SEP] Feed [SEP] Food [SEP]...,"[1.058161729698188, -0.49520412160069144, 0.0,...",1.0
3,Element Code [SEP] 5142 [SEP] 5521 [SEP] 5142 ...,"[1.058161729698188, -0.49520412160069144, 0.0,...",1.0
4,Item [SEP] Wheat and products [SEP] Rice (Mill...,"[1.058161729698188, -0.49520412160069144, 0.0,...",1.0


In [4]:
x_train, x_val, y_train, y_val = train_test_split(train_data[['text', "features"]], train_data['label'], 
                                                                    random_state=2018, 
                                                                    test_size=0.2, 
                                                                    stratify=train_data['label'])

In [5]:
transformer = AutoModel.from_pretrained('bert-base-uncased').to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
model_name = "berta_random_forest"

In [7]:
device = torch.device("cuda")

In [9]:
for param in transformer.parameters():
    param.requires_grad = False

In [8]:
train_dataloader, val_dataloader, test_dataloader = init_dataloaders(x_train, y_train, x_val, y_val, 
                                                                     test_data, model="bert")

/home/tamittal/.local/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2198: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [9]:
def rf_featurize(dataloader):
    X = []
    Y = []
    for step,batch in enumerate(dataloader):
            # push the batch to gpu
            batch = [r.to(device) for r in batch]

            sent_id, mask, features, labels = batch

            tf_out = transformer(sent_id, attention_mask=mask)

            for i in range(tf_out.pooler_output.shape[0]):
                emb = tf_out.pooler_output[i].tolist()
                feat = features[i].tolist()
                X.append(emb+feat)
                Y.append(int(labels[i]))
    X = np.asarray(X)
    Y = np.asarray(Y)
    return X, Y

In [10]:
X_train, Y_train = rf_featurize(train_dataloader)
X_val, Y_val = rf_featurize(val_dataloader)
X_test, Y_test = rf_featurize(test_dataloader)

In [11]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
clf = RandomForestClassifier()

In [12]:
clf.fit(X_train, Y_train)

RandomForestClassifier()

In [13]:
from sklearn.metrics import accuracy_score

In [14]:
print("train:", accuracy_score(Y_train, clf.predict(X_train)))
print("val:", accuracy_score(Y_val, clf.predict(X_val)))
print("test:", accuracy_score(Y_test, clf.predict(X_test)))

train: 1.0
val: 0.8664146187775678
test: 0.8181360201511335


In [15]:
model = Transformer_cnn(transformer, 256, [1, 2, 3, 4, 5])

model = model.to(device)

In [16]:
model.load_state_dict(torch.load("../models/exports/bert_cnn_256_1_2_3_4_5_best_924.pt"))

<All keys matched successfully>

In [17]:
def rf_featurize(dataloader):
    X = []
    Y = []
    for step,batch in enumerate(dataloader):
            # push the batch to gpu
            batch = [r.to(device) for r in batch]

            sent_id, mask, features, labels = batch

            preds, conv_out = model(sent_id, mask, features)

            for i in range(conv_out.shape[0]):
                X.append(conv_out[i].tolist())
                Y.append(int(labels[i]))
    X = np.asarray(X)
    Y = np.asarray(Y)
    return X, Y

In [18]:
X_train, Y_train = rf_featurize(train_dataloader)
X_val, Y_val = rf_featurize(val_dataloader)
X_test, Y_test = rf_featurize(test_dataloader)

In [21]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [24]:
clf = SVC()
clf.fit(X_train, Y_train)

SVC()

In [25]:
print("train:", accuracy_score(Y_train, clf.predict(X_train)))
print("val:", accuracy_score(Y_val, clf.predict(X_val)))
print("test:", accuracy_score(Y_test, clf.predict(X_test)))

train: 0.9604537576807941
val: 0.9010712035286704
test: 0.8977329974811084
